In [29]:
import pandas_datareader.data as web
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yfin
import datetime
import pandas as pd
yfin.pdr_override()

## Portfolios Return

In [2]:
end=datetime.date.today()
start=end-datetime.timedelta(365*5)
start,end

(datetime.date(2018, 7, 11), datetime.date(2023, 7, 10))

In [3]:
prices=web.DataReader(["LT.NS","TCS.NS","ICICIBANK.NS"],start,end)["Adj Close"]

[*********************100%***********************]  3 of 3 completed


In [4]:
prices.head()

,ICICIBANK.NS,LT.NS,TCS.NS
Date,,,
2018-07-11,260.975128,1163.524048,1778.277466
2018-07-12,264.422455,1186.342773,1770.598145
2018-07-13,260.004089,1185.110474,1779.355469
2018-07-16,251.750000,1171.145630,1786.630371
2018-07-17,258.741760,1174.568359,1801.885376


Assumption: Bough 100 shares of each 5 years ag0

In [5]:
#Initial Prices

initial_LT= prices["LT.NS"][0]
initial_TCS= prices["TCS.NS"][0]
initial_ICICIBANK= prices["ICICIBANK.NS"][0]

#Final Prices
final_LT= prices["LT.NS"][-1]
final_TCS= prices["TCS.NS"][-1]
final_ICICIBANK= prices["ICICIBANK.NS"][-1]

#Weights
weight_LT= initial_LT/(initial_ICICIBANK+initial_LT+initial_TCS)
weight_TCS= initial_TCS/(initial_ICICIBANK+initial_LT+initial_TCS)
weight_ICICIBANK= initial_ICICIBANK/(initial_ICICIBANK+initial_LT+initial_TCS)

In [6]:
print("We have initial investmetn in ICICIBANK Of %.3f, in LT of %.3f, and in TCS of %.3f"
      %(initial_ICICIBANK*100,initial_LT*100,initial_TCS*100)
     )

We have initial investmetn in ICICIBANK Of 26097.513, in LT of 116352.405, and in TCS of 177827.747


In [7]:
print(
    "The weight of ICICIBANK in total portfolio is %.3f, of LT is %.3f, and of TCS is %.3f"
    %(weight_ICICIBANK,weight_LT,weight_TCS)
)

The weight of ICICIBANK in total portfolio is 0.081, of LT is 0.363, and of TCS is 0.555


In [8]:
# Return Calculation
return_ICICIBANK=100*(final_ICICIBANK-initial_ICICIBANK)/initial_ICICIBANK
return_LT=100*(final_LT-initial_LT)/initial_LT
return_TCS=100*(final_TCS-initial_TCS)/initial_TCS


In [9]:
print(
    "The overall return of ICICIBANK is %.2f%%, of LT is %.2f%%, and of TCS is %.2f%%"
    %(return_ICICIBANK,return_LT,return_TCS)
)

The overall return of ICICIBANK is 262.64%, of LT is 110.51%, and of TCS is 87.22%


In [10]:
print(
    "The overall return of portfolio is %.2f%%"
    %(weight_ICICIBANK*return_ICICIBANK+weight_LT*return_LT+weight_TCS*return_TCS)
)

The overall return of portfolio is 109.97%


## Portfolio Variance

In [11]:
weights= np.array([weight_ICICIBANK,weight_LT,weight_TCS])
returns=prices.pct_change()
covariance=25*returns.cov()

In [12]:
variance=np.dot(weights.T,np.dot(covariance,weights))

print(str(np.round(variance,6)*100)+"%")

0.4411%


### Comparing Portfolio Variance with each individual variance

In [13]:
returns.var()*252

ICICIBANK.NS    0.116216
LT.NS           0.081026
TCS.NS          0.063909
dtype: float64

The variance of the portfolio is very low than that of the individual equities

**Standard Deviation**

In [14]:
np.round(math.sqrt(variance)*100,2)

6.64

The Standard Deviation of the portfolio is around 6.65%

### Sharpe ratio of the portfolio

In [15]:
sharpe= np.dot(weights.T,[return_ICICIBANK,return_LT,return_TCS])/5/6.65

#divided  by 5 to get approximate 1 year return
sharpe

3.3074905023283616

## Writing Reusable codes for all above

In [22]:
#Calculating Returns

def getReturns(starttime,endtime,tickers):
    prices=web.DataReader(list(tickers.keys()),starttime,endtime)["Adj Close"]
    prices=prices.rename(columns=tickers)
    prices=prices.dropna()
    return prices.pct_change()

In [23]:
ret= getReturns(start,end,{"ICICIBANK.NS":"ICICIBANK","LT.NS":"LT","TCS.NS":"TCS"})
ret

[*********************100%***********************]  3 of 3 completed


,ICICIBANK,LT,TCS
Date,,,
2018-07-11,NaN,NaN,NaN
2018-07-12,0.013209,0.019612,-0.004318
2018-07-13,-0.016709,-0.001039,0.004946
2018-07-16,-0.031746,-0.011784,0.004089
2018-07-17,0.027772,0.002922,0.008538
...,...,...,...
2023-07-03,0.008506,-0.008685,-0.009070
2023-07-04,0.002281,0.003178,0.011170
2023-07-05,0.014079,0.003635,0.003355


In [24]:
ret.corr()

,ICICIBANK,LT,TCS
ICICIBANK,1.000000,0.550685,0.282080
LT,0.550685,1.000000,0.273115
TCS,0.282080,0.273115,1.000000


In [41]:
def CompareVariance(starttime,endtime,tickers,weight):
    returns=getReturns(starttime,endtime,tickers)
    tmp= weights*returns
    returns["Portfolio"]= tmp[tmp.columns[0]]+tmp[tmp.columns[1]]+tmp[tmp.columns[2]]
    std=returns.std()
    avgRet=returns.mean()
    res=pd.concat([avgRet*100,std],axis=1)
    res.columns=["Daily Avg Return","Std return"]
    return res.round(3)

In [42]:
CompareVariance(start,end,{"ICICIBANK.NS":"ICICIBANK","LT.NS":"LT","TCS.NS":"TCS"},[0.4,0.4,0.2])

[*********************100%***********************]  3 of 3 completed


,Daily Avg Return,Std return
ICICIBANK,0.128,0.021
LT,0.077,0.018
TCS,0.064,0.016
Portfolio,0.074,0.013
